## Imports

In [159]:
import pandas as pd 
import numpy as np 
import requests
from selenium import webdriver
import matplotlib.pyplot as plt
import folium
from math import radians, sin, cos, asin, sqrt, ceil
import time
import os

from selenium import webdriver
import chromedriver_binary

from selenium.webdriver.common.keys import Keys

from selenium.webdriver.common.by import By
from selenium.webdriver.support.ui import WebDriverWait
from selenium.webdriver.support import expected_conditions as ec
from selenium.webdriver.chrome.options import Options

from bs4 import BeautifulSoup

from dotenv import load_dotenv, find_dotenv

In [160]:
path = find_dotenv()
load_dotenv(path)
yelp_key=os.getenv('YELP_KEY')
google_key=os.getenv('GOOGLE_PLACE_KEY')

## Get list of restaurants from Yelp API

In [161]:
#Haversine Distance function used to compute radius

def haversine_distance(lon1, lat1, lon2, lat2):
    """
    Compute distance between two pairs of coordinates (lon1, lat1, lon2, lat2)
    """
    lon1, lat1, lon2, lat2 = map(radians, [lon1, lat1, lon2, lat2])
    dlon = lon2 - lon1
    dlat = lat2 - lat1
    a = sin(dlat / 2) ** 2 + cos(lat1) * cos(lat2) * sin(dlon / 2) ** 2
    return 2 * 6371 * asin(sqrt(a))

In [162]:
#Divide Paris into subzones to request more restaurants from Yelp API

def subzones_paris(n_subzones):
    """
    Divide Paris into n_subzones squares. Returns list of centers (lat,lon)
    and the radius (in meters) to use for the API request
    """
    #lat-lon boundaries for Paris
    up_left=np.array([48.9002,2.2617])
    up_right=np.array([48.9002,2.4488])
    down_right=np.array([48.8202,2.4488])
    down_left=np.array([48.8202,2.2617])

    coor = np.array([up_left,up_right,down_right,down_left])

    #compute distance
    dist_h=up_right-up_left
    dist_v=up_left-down_left

    #divide in n_subzones

    epsilon_h=dist_h/n_subzones**0.5
    epsilon_v=dist_v/n_subzones**0.5

    centers=[]
    start=up_left

    for i in range(int(n_subzones**0.5)):
        for j in range(int(n_subzones**0.5)):
            start=(i+0.5)*epsilon_v+(j+0.5)*epsilon_h+up_left
            centers.append(start)
            
    
    radius = haversine_distance(up_left[0], up_left[1], centers[0][0], centers[0][1])
    
    return centers,radius*1000

In [163]:
def map_centers(centers,radius):
    centers_start=centers[int((len(centers)-1)/2)]
    m = folium.Map(location=list(centers_start), zoom_start=13)
    
    for c in centers:
        folium.Circle(radius=radius,location=list(c),
                      color="crimson",fill=False).add_to(m)
        
    return m 

In [164]:
map_centers(*subzones_paris(16))

In [165]:
def get_restaurants(centers,radius,api_key):
    """
    Returns DataFrame of restaurants in Paris
    """
    headers={'Authorization':f'Bearer {yelp_key}'}
    url = 'https://api.yelp.com/v3/businesses/search'

    data = []
    
    for i,c in enumerate(centers):
        print(f'---------- Requesting API for subzone #{i+1} ----------')
        for offset in range(0, 200, 50):
            print(f'   ------- Requesting API with offset = {offset} -------   ')
            params = {
                'limit': 50, 
                'categories':['restaurants'],
                'sort_by':'review_count',
                'offset': offset,
                'latitude':c[0],
                'longitude':c[1],
                'radius':int(radius)
            }

            response = requests.get(url, headers=headers, params=params)
            if response.status_code == 200:
                data += response.json()['businesses']
            elif response.status_code == 400:
                print('400 Bad Request')
                break
                
    print(f'#####   Request completed, {len(data)} businesses fetched   ###')
    return data

In [166]:
centers,radius = subzones_paris(4)
data = get_restaurants(centers,radius,yelp_key)

---------- Requesting API for subzone #1 ----------
   ------- Requesting API with offset = 0 -------   
   ------- Requesting API with offset = 50 -------   
   ------- Requesting API with offset = 100 -------   
   ------- Requesting API with offset = 150 -------   
---------- Requesting API for subzone #2 ----------
   ------- Requesting API with offset = 0 -------   
   ------- Requesting API with offset = 50 -------   
   ------- Requesting API with offset = 100 -------   
   ------- Requesting API with offset = 150 -------   
---------- Requesting API for subzone #3 ----------
   ------- Requesting API with offset = 0 -------   
   ------- Requesting API with offset = 50 -------   
   ------- Requesting API with offset = 100 -------   
   ------- Requesting API with offset = 150 -------   
---------- Requesting API for subzone #4 ----------
   ------- Requesting API with offset = 0 -------   
   ------- Requesting API with offset = 50 -------   
   ------- Requesting API with off

## Yelp Reviews Scrapping

In [11]:
### Create DF for Yelp data


def create_df_yelp(data):

    df = pd.DataFrame(columns=[
        'alias', 'name', 'url', 'categories', 'latitude', 'longitude',
        'address', 'zip_code', 'price', 'rating', 'review_count'
    ])

    features_to_loop = [
        'alias', 'name', 'url', 'categories', 'price', 'rating', 'review_count'
    ]

    #populate DF
    #if condition to avoid raising errors in case restaurant doesn't have all informations

    for i, d in enumerate(data):

        for f in features_to_loop:
            if f in d:
                df.loc[i, f] = d[f]
            else:
                df.loc[i, f] = ''

        if 'location' in d:
            if 'latitude' in d['coordinates']:
                df.loc[i, 'latitude'] = d['coordinates']['latitude']
            else:
                df.loc[i, 'latitude'] = ''

            if 'longitude' in d['coordinates']:
                df.loc[i, 'longitude'] = d['coordinates']['longitude']
            else:
                df.loc[i, 'longitude'] = ''

            if 'address1' in d['location']:
                df.loc[i, 'address'] = d['location']['address1']
            else:
                df.loc[i, 'address'] = ''

            if 'zip_code' in d['location']:
                df.loc[i, 'zip_code'] = d['location']['zip_code']
            else:
                df.loc[i, 'zip_code'] = 0

    #clean DF
    #dtypes
    df['latitude'] = df['latitude'].astype(float)
    df['longitude'] = df['longitude'].astype(float)
    df['zip_code'] = df['zip_code'].replace('', 0).astype(int)
    df['rating'] = df['rating'].astype(float)
    df['review_count'] = df['review_count'].astype(float)

    #url
    df['url'] = df['url'].apply(lambda txt: txt.split('?', 1)[0])

    #price
    prices = {'€': '1', '€€': '2', '€€€': '3', '€€€€': '4'}

    for euro, num in prices.items():
        df['price'] = df['price'].replace(euro, num)

    df['price'] = df['price'].replace('', 0).astype(int)

    #categories
    df['categories'] = df['categories'].apply(
        lambda dicts: ', '.join([d['alias'] for d in dicts]))

    return df.drop_duplicates()

In [12]:
df = create_df_yelp(data)

In [13]:
df.head()

,alias,name,url,categories,latitude,longitude,address,zip_code,price,rating,review_count
0,bouillon-chartier-paris,Bouillon Chartier,https://www.yelp.com/biz/bouillon-chartier-paris,french,48.871940,2.343170,7 rue du Faubourg Montmartre,75009,2,3.5,953.0
1,le-potager-du-père-thierry-paris-2,Le Potager du Père Thierry,https://www.yelp.com/biz/le-potager-du-p%C3%A8...,french,48.884476,2.341335,16 rue des Trois Frères,75018,2,4.5,427.0
2,la-cave-gourmande-paris-2,La Cave Gourmande,https://www.yelp.com/biz/la-cave-gourmande-par...,bistros,48.884499,2.339847,96 rue des Martyrs,75018,2,4.5,336.0
3,holybelly-19-paris,Holybelly 19,https://www.yelp.com/biz/holybelly-19-paris,"breakfast_brunch, cafes, desserts",48.872130,2.360780,19 rue Lucien Sampaix,75010,2,4.5,296.0
4,le-relais-de-venise-l-entrecôte-paris-2,Le Relais de Venise - l'Entrecôte,https://www.yelp.com/biz/le-relais-de-venise-l...,"desserts, bbq",48.878045,2.285062,271 boulevard Pereire,75017,3,4.0,275.0


In [14]:
df.loc[50]

alias                                    la-fourmi-paris
name                                           La Fourmi
url             https://www.yelp.com/biz/la-fourmi-paris
categories                                         cafes
latitude                                         48.8823
longitude                                        2.33965
address                               74 rue des Martyrs
zip_code                                           75018
price                                                  1
rating                                               3.5
review_count                                       126.0
Name: 50, dtype: object

In [15]:
url_ = df.loc[50,'url']

In [84]:
k =[]
x=3
for i in range(10):
    try:
        kaka
        k.append(i)
    except:
        k.append(0)
    

In [120]:
### SCRAPPING ONE RESTAURANT --- YELP ###

def get_reviews_yelp(url,verbose=0,quiet_mode=True,load_strategy='eager'):
    
    #pages
    
    options=Options()
    if quiet_mode:
        options.add_argument('--headless')
    options.page_load_strategy = load_strategy
    driver = webdriver.Chrome(options=options)
    
    driver.get(url)
    xpath_page="//yelp-react-root/div[1]/div[4]/div/div/div[2]/div/div[1]/div[2]/section/div[2]/div/div[4]/div[2]/span"
    try:
        page = driver.find_element_by_xpath(xpath_page).text
        n_pages=int(page.split('of ')[1])
        reviews=[]
        rates=[]
        dates=[]

        print(f'### {n_pages} pages to scrap ###')

        for n in range(n_pages):

            if verbose>0:
                print(f'--- Fetching reviews of page #{n+1}...')

            #url of the n_page
            url_=f'{url}?start={n*10}'
            driver.get(url_)
            xpath_all_review=f"//yelp-react-root/div[1]/div[4]/div/div/div[2]/div/div[1]/div[2]/section/div[2]/div/ul"   

            review_block = driver.find_element(By.XPATH,xpath_all_review)
            all_reviews = review_block.find_elements(By.TAG_NAME,'li')
            for i,r in enumerate(all_reviews):
                if verbose>1:
                    print(f'--Getting reviews #{i+1}...')
                reviews.append(r.text)
                rate=r.find_element(By.XPATH,'.//div/div[2]/div/div[1]/span/div')
                rates.append(rate.get_attribute('aria-label'))
                dates.append(r.find_element(By.XPATH,'.//div/div[2]/div/div[2]/span').text)
    except:
        print('error pages')
        
    return dates, rates, reviews


In [121]:
dates_bis, rates_bis, reviews_bis  = [],[],[]
dates, rates, reviews = [],[],[]
for i in range(0,3):
    dates, rates, reviews  = get_reviews_yelp(df.loc[i,'url'],verbose=2, load_strategy='normal')
    if len(rates) == len(reviews):
        dates_bis+=dates
        rates_bis+=rates
        reviews_bis+=reviews
    else:
        for i in range(len(rates)):
            reviews_bis.append(np.nan)

/var/folders/g2/fbwf804j5jq9lyct2w68nmvc0000gn/T/ipykernel_9244/2017355025.py:16: DeprecationWarning: find_element_by_* commands are deprecated. Please use find_element() instead
  page = driver.find_element_by_xpath(xpath_page).text


### 33 pages to scrap ###
--- Fetching reviews of page #1...
--Getting reviews #1...
--Getting reviews #2...
--Getting reviews #3...
--Getting reviews #4...
--Getting reviews #5...
--Getting reviews #6...
--Getting reviews #7...
--Getting reviews #8...
--Getting reviews #9...
--Getting reviews #10...
--- Fetching reviews of page #2...
--Getting reviews #1...
--Getting reviews #2...
--Getting reviews #3...
--Getting reviews #4...
--Getting reviews #5...
--Getting reviews #6...
--Getting reviews #7...
--Getting reviews #8...
--Getting reviews #9...
--Getting reviews #10...
--- Fetching reviews of page #3...
--Getting reviews #1...
--Getting reviews #2...
--Getting reviews #3...
--Getting reviews #4...
--Getting reviews #5...
--Getting reviews #6...
--Getting reviews #7...
--Getting reviews #8...
--Getting reviews #9...
--Getting reviews #10...
--- Fetching reviews of page #4...
--Getting reviews #1...
--Getting reviews #2...
--Getting reviews #3...
--Getting reviews #4...
--Getting revie

--Getting reviews #6...
--Getting reviews #7...
--Getting reviews #8...
--Getting reviews #9...
--Getting reviews #10...
--- Fetching reviews of page #31...
--Getting reviews #1...
--Getting reviews #2...
--Getting reviews #3...
--Getting reviews #4...
--Getting reviews #5...
--Getting reviews #6...
--Getting reviews #7...
--Getting reviews #8...
--Getting reviews #9...
--Getting reviews #10...
--- Fetching reviews of page #32...
--Getting reviews #1...
--Getting reviews #2...
--Getting reviews #3...
--Getting reviews #4...
--Getting reviews #5...
--Getting reviews #6...
--Getting reviews #7...
--Getting reviews #8...
--Getting reviews #9...
--Getting reviews #10...
--- Fetching reviews of page #33...
--Getting reviews #1...


/var/folders/g2/fbwf804j5jq9lyct2w68nmvc0000gn/T/ipykernel_9244/2017355025.py:16: DeprecationWarning: find_element_by_* commands are deprecated. Please use find_element() instead
  page = driver.find_element_by_xpath(xpath_page).text


### 32 pages to scrap ###
--- Fetching reviews of page #1...
--Getting reviews #1...
--Getting reviews #2...
--Getting reviews #3...
--Getting reviews #4...
--Getting reviews #5...
--Getting reviews #6...
--Getting reviews #7...
--Getting reviews #8...
--Getting reviews #9...
--Getting reviews #10...
--- Fetching reviews of page #2...
--Getting reviews #1...
--Getting reviews #2...
--Getting reviews #3...
--Getting reviews #4...
--Getting reviews #5...
--Getting reviews #6...
--Getting reviews #7...
--Getting reviews #8...
--Getting reviews #9...
--Getting reviews #10...
--- Fetching reviews of page #3...
--Getting reviews #1...
--Getting reviews #2...
--Getting reviews #3...
--Getting reviews #4...
--Getting reviews #5...
--Getting reviews #6...
--Getting reviews #7...
--Getting reviews #8...
--Getting reviews #9...
--Getting reviews #10...
--- Fetching reviews of page #4...
--Getting reviews #1...
--Getting reviews #2...
--Getting reviews #3...
--Getting reviews #4...
--Getting revie

KeyboardInterrupt: 

In [156]:
dic_df = {'Rating': rates_bis, 'Dates':dates_bis, 'Reviews':reviews_bis}


In [158]:
pd.DataFrame(dic_df).to_csv('review_yelp.csv')

In [125]:
options=Options()
if True:
    options.add_argument('--headless')
options.page_load_strategy = 'normal'
driver = webdriver.Chrome(options=options)
    
driver.get(url_)
reviews=[]
for i in range(1,11):
    xpath_review=f"//yelp-react-root/div[1]/div[4]/div/div/div[2]/div/div[1]/div[2]/section/div[2]/div/ul"
    review = driver.find_element_by_xpath(xpath_review)
    reviews.append(review)
reviews

/var/folders/zd/9ts3jb2s5472mty2sgdddp3m0000gn/T/ipykernel_42052/2119553630.py:11: DeprecationWarning: find_element_by_* commands are deprecated. Please use find_element() instead
  review = driver.find_element_by_xpath(xpath_review)


AttributeError: 'list' object has no attribute 'shape'

## Google Review Scrapping

In [35]:
## Get place ID 

def get_place_google_id(name,latitude,longitude):

    url = 'https://maps.googleapis.com/maps/api/place/findplacefromtext/json'
    params={
        'key' : google_key,
        'input' : name,
        'inputtype' : 'textquery',
        'locationbias' : f'point:{latitude},{longitude}'
    }
    
    response = requests.get(url,params=params)
    
    #if conditions to avoid raising errors
    if response.status_code != 200:
        return ''
        
    if 'candidates' in response.json():
        response = response.json()['candidates']
        if len(response)==0:
            return ''
        if 'place_id' in response[0]:
            return response[0]['place_id']
    
    return ''


In [36]:
## id_ for testing purposes -- don't request API multiple times

name = df.loc[0,'name']
lat = df.loc[0,'latitude']
lon = df.loc[0,'longitude']
id_=get_place_google_id(name,lat,lon)

id_

'ChIJ4xutfT5u5kcRaJn2NkiOhPU'

In [37]:
## Get place url

def get_place_google_url(place_id):
    url='https://maps.googleapis.com/maps/api/place/details/json'
    params={
        'key' : google_key,
        'place_id' : place_id,
        'fields' : 'url'
    }
    
    response = requests.get(url,params=params)
    
    #if conditions to avoid raising errors
    if response.status_code != 200:
        return ''
    
    if 'result' in response.json():
        response = response.json()['result']
        if 'url' in response:
            return response['url']
    
    return ''


In [38]:
## url_for_test for testing purposes -- don't request API multiple times

url_for_test=get_place_google_url(id_)
url_for_test

'https://maps.google.com/?cid=17691421677029071208'

In [39]:
### Get all reviews from a Google page


def get_reviews_google(url,scroll_limit=None,quiet_mode=True,return_count=False):
    
    options=Options()
    if quiet_mode:
        options.add_argument('--headless')    
    driver = webdriver.Chrome(options=options)
    driver.get(url)

    
    ###Expand all the reviews using Selenium
        # privacy pop-up
    xpath = "/html/body/c-wiz/div/div/div/div[2]/div[1]/div[4]/form/div[1]/div/button/span"
    driver.find_element_by_xpath(xpath).click()

        #review_count click
    xpath = '//*[@id="pane"]/div/div[1]/div/div/div[2]/div[1]/div[1]/div[2]/div/div[1]/span[1]/span/span[1]/span[2]'

    review_count = driver.find_element_by_xpath(xpath).text
    review_count=review_count.split(' ', 1)[0]

    driver.find_element_by_xpath(xpath).click()

        #scroll to show all reviews
    time.sleep(2)
    if scroll_limit:
        review_count=scroll_limit
    scrollable_div = driver.find_element_by_xpath('//*[@id="pane"]/div/div[1]/div/div/div[2]')
    for i in range(0,(round(int(review_count)/10-1))):
            driver.execute_script('arguments[0].scrollTop = arguments[0].scrollHeight', 
                    scrollable_div)
            time.sleep(2)


    ### Scrap the reviews info using BS      
    soup = BeautifulSoup(driver.page_source, 'html.parser')
    
    #Scrap the reviews text
    reviews_soup = soup.find_all('div', class_='ODSEW-ShBeI NIyLF-haAclf gm2-body-2')
    reviews = [r.text for r in reviews_soup]
    
    #Scrap the reviews rate
    review_rates_soup = [s.find('span',class_='ODSEW-ShBeI-H1e3jb') for s in reviews_soup]
    review_rates = [rr.attrs['aria-label'][1] for rr in review_rates_soup]
    
    #Scrap the reviews date
    review_dates_soup=[s.find('span', class_='ODSEW-ShBeI-RgZmSc-date') for s in reviews_soup]
    review_dates=[rd.text for rd in review_dates_soup]
    
    
    if return_count:
        return review_count,review_dates,review_rates,reviews
    
    return review_dates,review_rates,reviews


In [40]:
review_dates,review_rates,reviews = get_reviews_google(url_for_test,scroll_limit=40,quiet_mode=False)

/var/folders/zd/9ts3jb2s5472mty2sgdddp3m0000gn/T/ipykernel_42052/2372629914.py:16: DeprecationWarning: find_element_by_* commands are deprecated. Please use find_element() instead
  driver.find_element_by_xpath(xpath).click()
/var/folders/zd/9ts3jb2s5472mty2sgdddp3m0000gn/T/ipykernel_42052/2372629914.py:21: DeprecationWarning: find_element_by_* commands are deprecated. Please use find_element() instead
  review_count = driver.find_element_by_xpath(xpath).text
/var/folders/zd/9ts3jb2s5472mty2sgdddp3m0000gn/T/ipykernel_42052/2372629914.py:24: DeprecationWarning: find_element_by_* commands are deprecated. Please use find_element() instead
  driver.find_element_by_xpath(xpath).click()
/var/folders/zd/9ts3jb2s5472mty2sgdddp3m0000gn/T/ipykernel_42052/2372629914.py:30: DeprecationWarning: find_element_by_* commands are deprecated. Please use find_element() instead
  scrollable_div = driver.find_element_by_xpath('//*[@id="pane"]/div/div[1]/div/div/div[2]')


In [41]:
review_dates

['il y a une semaine',
 'il y a 4\xa0jours',
 'il y a 4\xa0jours',
 'il y a un mois',
 'il y a un mois',
 'il y a 2\xa0mois',
 'il y a 2\xa0mois',
 'il y a une semaine',
 'il y a un mois',
 'il y a 2\xa0mois',
 'il y a 3\xa0mois',
 'il y a 2\xa0mois',
 'il y a 3\xa0mois',
 'il y a 3\xa0mois',
 'il y a 4\xa0mois',
 'il y a 3\xa0semaines',
 'il y a 4\xa0mois',
 'il y a 3\xa0mois',
 'il y a 4\xa0mois',
 'il y a 2\xa0semaines',
 'il y a une semaine',
 'il y a une semaine',
 'il y a une semaine',
 'il y a un mois',
 'il y a un mois',
 'il y a 4\xa0mois',
 'il y a 3\xa0semaines',
 'il y a 3\xa0mois',
 'il y a un mois',
 'il y a 3\xa0mois',
 'il y a une semaine',
 'il y a une semaine',
 'il y a 3\xa0mois',
 'il y a un mois',
 'il y a 4\xa0mois',
 'il y a une semaine',
 'il y a 3\xa0mois',
 'il y a 3\xa0semaines',
 'il y a une semaine',
 'il y a 3\xa0mois']

In [42]:
reviews

["                  Jean-Luc Paredes  Local\xa0Guide6\xa0avis               il y a une semaine Nouveau    Excellent ! Typique ! La vrai brasserie parisienne telle qu'on l'espère ! Un serveur au top. Des voisins de table sympa. Tout était réuni pour passer un agréable moment !       Visité en novembre                J'aime    Partager       ",
 "                  Pascal vacaresse  Local\xa0Guide · 48\xa0avis               il y a 4\xa0jours Nouveau    De passage à Paris pour quelques jours. Une visite dans ce lieu me semblait incontournable. Victime de leur succès 1h30 d'attente pour avoir une table. Restauration convenable et les prix accessibles pour tous. Un peu bruyant. …    Plus      Visité en novembre             +2         J'aime    Partager       ",
 "                  Marie Claude Suchet  Local\xa0Guide · 172\xa0avis               il y a 4\xa0jours Nouveau    C'est vrai c'est vraiment une brasserie hors du temps où vous pouvez manger seul, car c'est un véritable spectacle. Nous 

In [43]:
data_to_clean = pd.DataFrame({'data':review_dates,
                              'rate':review_rates,
                              'review':reviews})
data_to_clean.head()

,data,rate,review
0,il y a une semaine,4,Jean-Luc Paredes Local Guid...
1,il y a 4 jours,4,Pascal vacaresse Local Guid...
2,il y a 4 jours,5,Marie Claude Suchet Local G...
3,il y a un mois,4,Michel C. Local Guide · 140...
4,il y a un mois,5,Jackie Ibanez Local Guide ·...


In [44]:
data_to_clean.to_csv('data_to_clean.csv')